<a href="https://colab.research.google.com/github/ndeekshith/Titanic_data_analysis/blob/main/Titanic_data_analysis_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'titanic-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1818188%2F2965537%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20250626%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20250626T123603Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D574574c78c6ee3564f6f3ac277af0b752dc022b019515f4131dfa4ee9a3137d7c45119ae562193e6545043f3afbd478bd4a1ec17353ab0294a6121d459995dec8c6baa7dba7b9e12cd5e71e1f3a550ea221932487ff03790fb1bd755a41d7db8cc39cd26d98ade8e748a68b532b07ef46108307e9015446c160931dffc85a525a1248756871f1c93649b7af529309514dc088c2807e8272b5962371e0abca873074f9b6a3f6454f412e24a125216a98ec0e4b708566fdb890914ed6d23c190d4d9a1561fe07e3e8b5a152970480e3aa2aeb848ebee719c793a3ab37a8c1adec4684042d87e75fbe0ebd7bd9761fbcf9d20b78f368e3fdea6ede77736caaadb84'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load all the libraries
library(tidyverse)
library(caret)
library(randomForest)
library(e1071)
library(rpart)
library(rpart.plot)
library(corrplot)
library(VIM)
library(mice)
library(ggplot2)
library(gridExtra)

In [ ]:
titanic_data <- read.csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

In [ ]:
# Convert categorical variables into factors
titanic_data$Survived <- as.factor(titanic_data$Survived)
titanic_data$Pclass <- as.factor(titanic_data$Survived)
titanic_data$Sex <- as.factor(titanic_data$Sex)
titanic_data$Embarked <- as.factor(titanic_data$Embarked)

In [ ]:
# Data Analysis

cat("Number of passengers :",nrow(titanic_data),"\n")
cat("Number of variables:",ncol(titanic_data),"\n\n")

In [ ]:
#Structure of data
cat("Dataset structure")
str(titanic_data)

In [ ]:
# Summary of the data
summary(titanic_data)

In [ ]:
# Missing value analysis
missing_values <- sapply(titanic_data,function(x)sum(is.na(x)))
missing_percent <- round(100 * missing_values / nrow(titanic_data), 2)
missing_df <- data.frame(
  Variables = names(missing_values),
  Missing_count <- missing_values,
  Missing_Percent <- missing_percent
)
print(missing_df[missing_df$Missing_count....missing_values > 0,])

In [ ]:
# Data visulation
# 1.Survival rate
p1 <- ggplot(titanic_data,aes(x = Survived,fill = Survived)) +geom_bar() + geom_text(stat = 'count', aes(label = `..count..`),vjust = -0.5) + labs(title = "Survival distribution", x = "Survived(0=No, 1=Yes)", y = "Count") + theme_minimal()
print(p1)

In [ ]:
#2.Survival by gender
p2 <- ggplot(titanic_data, aes(x = Sex, fill = Survived)) + geom_bar(position = "dodge") + labs(title = "Survival by Gender", x = "Gender", y = "Count") +  theme_minimal()
print(p2)

In [ ]:
#3.Survival by class
p3 <- ggplot(titanic_data, aes(x = Pclass, fill = Survived)) + geom_bar(position = "dodge") + labs(title = "Survival by passengers class", x = "Passengers class", y = "Count") + theme_minimal()
print(p3)


In [ ]:
# 4.Age Distribution
p4 <- ggplot(titanic_data[!is.na(titanic_data$Age), ], aes(x = Age, fill = Survived)) +  geom_histogram(bins = 10, alpha = 0.7, position = "identity") +  labs(title = "Age Distribution by Survival", x = "Age", y = "Count") +  theme_minimal()
print(p4)

In [ ]:
# Combine all the plots
grid.arrange(p1,p2,p3,p4, ncol = 2)

In [ ]:
# Data preprocessing
titanic_processed <- titanic_data
titanic_processed$Age[is.na(titanic_processed$Age)] <- median(titanic_processed$Age,na.rm = TRUE)

# Extract title from name
titanic_processed$Title <- gsub('(.*, )|(\\..*)', '', titanic_processed$Name)

# Age groups
titanic_processed$AgeGroup <- cut(titanic_processed$Age,
                                  breaks = c(0, 12, 18, 35, 60, 100),
                                  labels = c("Child", "Teen", "Adult", "Middle", "Senior"))

In [ ]:
# Select features for modeling
features_for_modeling <- c("Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked")
modeling_data <- titanic_processed[, c("Survived", features_for_modeling)]

In [ ]:
# Remove rows with missing values in selected features
modeling_data <- na.omit(modeling_data)

In [ ]:
cat("Number of complete cases:", nrow(modeling_data), "\n")

In [ ]:
# Train test splitting the data
train_index <- createDataPartition(modeling_data$Survived, p = 0.8, list = FALSE)
train_data <- modeling_data[train_index, ]
test_data <- modeling_data[-train_index, ]

cat("Training set size:", nrow(train_data), "\n")
cat("Test set size:", nrow(test_data), "\n")
cat("Training set proportion:", round(nrow(train_data)/nrow(modeling_data), 2), "\n")

In [ ]:
set.seed(123)

# Convert factors for some models
modeling_data_numeric <- modeling_data
modeling_data_numeric$Sex <- as.numeric(as.factor(modeling_data_numeric$Sex))
modeling_data_numeric$Embarked <- as.numeric(as.factor(modeling_data_numeric$Embarked))
modeling_data_numeric$Pclass <- as.numeric(modeling_data_numeric$Pclass)

In [ ]:
# Model 1 Logistic regression
logistic_model <- glm(Survived ~ ., data = modeling_data, family = binomial)
summary(logistic_model)

# Predictions
logistic_pred <- predict(logistic_model, type = "response")
logistic_pred_class <- ifelse(logistic_pred > 0.5, 1, 0)

# Confusion Matrix
logistic_cm <- table(Actual = modeling_data$Survived, Predicted = logistic_pred_class)
cat("\nLogistic Regression Confusion Matrix:\n")
print(logistic_cm)

logistic_accuracy <- sum(diag(logistic_cm)) / sum(logistic_cm)
cat("Accuracy:", round(logistic_accuracy, 3), "\n")

In [ ]:
# Model 2 Decision tree

tree_model <- rpart(Survived ~ ., data = modeling_data, method = "class")

# Plot the tree
rpart.plot(tree_model, main = "Decision Tree for Titanic Survival")

# Predictions
tree_pred <- predict(tree_model, type = "class")

# Confusion Matrix
tree_cm <- table(Actual = modeling_data$Survived, Predicted = tree_pred)
cat("\nDecision Tree Confusion Matrix:\n")
print(tree_cm)

tree_accuracy <- sum(diag(tree_cm)) / sum(tree_cm)
cat("Accuracy:", round(tree_accuracy, 3), "\n")

In [ ]:
# Model 3 Random Forest
rf_model <- randomForest(Survived ~ ., data = modeling_data, ntree = 100, importance = TRUE)
print(rf_model)

# Variable importance
cat("\nVariable Importance:\n")
importance_df <- data.frame(
  Variable = rownames(importance(rf_model)),
  Importance = importance(rf_model)[, "MeanDecreaseGini"]
)
importance_df <- importance_df[order(importance_df$Importance, decreasing = TRUE), ]
print(importance_df)

# Predictions
rf_pred <- predict(rf_model)

# Confusion Matrix
rf_cm <- table(Actual = modeling_data$Survived, Predicted = rf_pred)
cat("\nRandom Forest Confusion Matrix:\n")
print(rf_cm)

rf_accuracy <- sum(diag(rf_cm)) / sum(rf_cm)
cat("Accuracy:", round(rf_accuracy, 3), "\n")

In [ ]:
cat("\n=== KEY INSIGHTS ===\n")
cat("1. Gender was a strong predictor of survival (women had higher survival rates)\n")
cat("2. Passenger class influenced survival (higher class = better survival)\n")
cat("3. Age played a role (children had higher survival rates)\n")
cat("4. Family size affected survival chances\n")
cat("5. Random Forest achieved the highest accuracy on this dataset\n")

# Survival rates by key factors
cat("\nSurvival Rates by Key Factors:\n")
survival_by_sex <- aggregate(as.numeric(as.character(titanic_data$Survived)),
                             by = list(titanic_data$Sex),
                             FUN = mean, na.rm = TRUE)
names(survival_by_sex) <- c("Sex", "Survival_Rate")
print(survival_by_sex)
survival_by_class <- aggregate(as.numeric(as.character(titanic_data$Survived)),
                               by = list(titanic_data$Pclass),
                               FUN = mean, na.rm = TRUE)
names(survival_by_class) <- c("Pclass", "Survival_Rate")
print(survival_by_class)

